# Visualizing changing trends in ODI cricket - Analysing ball by ball data

One Day Internationals have been continously evolving. This analysis aims to look at how the trends in scoring have changed over the past few years (2006 - 2020). The data is obtained from Cricsheet (https://cricsheet.org). For this analysis only men's ODIs are considered due to the smaller sample available from women's ODIs. 

## 1. Importing Libraries

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml, json
import re
import math

# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
# import plotly.figure_factory as ff
import plotly.io as pio
from plotly.subplots import make_subplots

from datetime import datetime
from IPython.core.display import HTML

from urllib.request import urlopen
from bs4 import BeautifulSoup
import difflib 

# set up seaborn
# sns.set(style="whitegrid")

# SettingWithCopyWarning
# pd.options.mode.chained_assignment = 'warn'

# styling the notebook
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

## 2. Importing Data
### 2.1 Importing list of matches
First up, let us import the list of all matches - we will then import the ball by ball details of these matches. 

In [31]:
matches_df = pd.read_csv('odis/README.txt', header = None, skiprows = 23, sep = "-")
matches_df.columns = ['year', 'month', 'day', 'type', 'type_2', 'gender', 'match_id', 'teams']

len(matches_df[(matches_df.type == ' international ') & (matches_df.type_2 == ' ODI ')]) == len(matches_df)

True

We can drop both the type and type_2 columns. Also keeping only men's matches and only matches between top teams.

In [32]:
# keep only mens ODIs
matches_df = matches_df[matches_df.gender == ' male ']

# convert to datetime
matches_df['date'] = matches_df.apply(lambda row: datetime(row['year'], row['month'], row['day']), axis = 1)

# split teams
coltitle = matches_df['teams'].apply(lambda s: pd.Series({'team_1': s.split(' vs ')[0].strip(),
                                                          'team_2': s.split(' vs ')[1].strip()}))
matches_df = pd.concat([matches_df, coltitle], axis = 1) 

# remove matches without top teams
top_teams = ['Ireland', 'West Indies', 'India', 'Afghanistan', 'Sri Lanka', 'Pakistan', 
             'Bangladesh', 'New Zealand', 'England', 'Australia', 'Zimbabwe', 'South Africa']
matches_df = matches_df[(matches_df.team_1.isin(top_teams)) & (matches_df.team_2.isin(top_teams))]

# drop columns that are not required, ro-order and re-index
matches_df = matches_df.loc[::-1]
matches_df = matches_df.reset_index()
matches_df.drop(['index', 'type', 'type_2', 'gender', 'day', 'month', 'teams'], axis = 1, inplace = True)
matches_df = matches_df[['match_id', 'team_1', 'team_2', 'date', 'year']]

matches_df.head()

,match_id,team_1,team_2,date,year
0,226391,New Zealand,Sri Lanka,2006-01-03,2006
1,226392,New Zealand,Sri Lanka,2006-01-06,2006
2,226375,Australia,South Africa,2006-01-15,2006
3,226383,South Africa,Sri Lanka,2006-01-17,2006
4,226376,Australia,South Africa,2006-01-20,2006


Now, let's plot the number of matches per year.

In [33]:
# group by year to get counts
year_groups = matches_df['match_id'].groupby(matches_df.year).agg('count')

year_groups_df = pd.DataFrame(year_groups)
year_groups_df = year_groups_df.rename(columns = {"match_id": "count"})
year_groups_df['year'] = year_groups_df.index
year_groups_df['remainder'] = year_groups_df['year'] % 4

fig = go.Figure()

fig.add_trace(go.Bar(
    x = year_groups_df.loc[year_groups_df['remainder'] == 3, 'year'],
    y = year_groups_df.loc[year_groups_df['remainder'] == 3, 'count'],
    text = year_groups_df.loc[year_groups_df['remainder'] == 3, 'count'],
    textposition = 'inside',
    textfont = dict(color = 'white'),
    name = 'World Cup Years',
    marker_color = "#ff7f0e",
    hoverinfo = 'skip',
    width = 0.8
))

fig.add_trace(go.Bar(
    x = year_groups_df.loc[year_groups_df['remainder'] != 3, 'year'],
    y = year_groups_df.loc[year_groups_df['remainder'] != 3, 'count'],
    text = year_groups_df.loc[year_groups_df['remainder'] != 3, 'count'],
    textposition = 'inside',
    textfont = dict(color = 'white'),
    name = 'Other Years',
    marker_color = "#1f77b4",
    hoverinfo = 'skip',
    width = 0.8
))

fig.update_layout(
    xaxis = dict(title = 'Year', tickmode = 'linear'),
    yaxis = dict(title = 'Number of Matches'),
    template = 'plotly_white'
)

fig.show()

We can immediately see that 2018 is missing. Also, on deeper investigation, it can be seen that data is missing between July 2017 and February 2019. Looks odd, but continuing on. <br>
Another interesting point to note is that World Cup years tend to have more matches - which makes sense. 2019 has fewer matches only because the data is incomplete for 2019.
### 2.2 Importing the ball by ball details
Now, we can import the ball by ball details for all the above selected matches. <br>
This is where the formatting of data takes place. Each ball is read for necessary information and data is stored into a dataframe. Special care is taken for extras where balls are delivered again (wides, noballs) as events from both need to be considered. <br>
Information such as runs, wickets, extras, fours, sixes and bowler are stored. <br>
Further, matches dataframe is also updated with information regarding winner, toss and ground. 

In [34]:
def get_inning_df(deliveries, team, innings_name, match_id):

    innings_list = []       # empty list

    runs_from_prev = 0      # extra+actual runs from previous illegal balls which has to be added to the legal ball
    extras_from_prev = 0    # extras from previous illegal balls which has to be added to the legal ball
    wickets = 0             # wickets in that particular ball (can be more than one as wide+OUT and OUT in re-ball)
    fours = 0               # fours in that particular ball (can be more than one due to no-balls)
    sixes = 0               # sixes in that particular ball (can be more than one due to no-balls)
    runs_cume = 0           # total score
    wickets_cume = 0        # total wickets down
    ball_no = 0

    for delivery in deliveries:

        # runs scored in wide and noballs have to be added onto the next legal delivery
        wide_match = re.search(r"\'wides\':", str(delivery))
        noball_match = re.search(r"\'noballs\':", str(delivery))

        if wide_match is not None:
            run_match = re.search(r"\'total\': (\d)", str(delivery))
            runs_from_prev = runs_from_prev + int(run_match.group(1))
            
            extra_match = re.search(r"\'extras\': (\d)", str(delivery))
            extras_from_prev = extras_from_prev + int(extra_match.group(1))

            # wicket can fall on wide ball (stumping). So, possible to have more than one wicket for a legal ball
            wicket_match = re.search(r"\'wicket\':", str(delivery))
            if wicket_match is not None:
                wickets = wickets + 1

        elif noball_match is not None:
            run_match = re.search(r"\'total\': (\d)", str(delivery))
            runs_from_prev = runs_from_prev + int(run_match.group(1))
            
            extra_match = re.search(r"\'extras\': (\d)", str(delivery))
            extras_from_prev = extras_from_prev + int(extra_match.group(1))
            
            four_match = re.search(r"\'batsman\': 4", str(delivery))
            if four_match is not None:
                fours = fours + 1
            
            six_match = re.search(r"\'batsman\': 6", str(delivery))
            if six_match is not None:
                sixes = sixes + 1

        # anything other than wide and noball (no re-balls for other extras)
        else:
            wicket_match = re.search(r"\'wicket\':", str(delivery))
            if wicket_match is not None:
                wickets = wickets + 1
            
            four_match = re.search(r"\'batsman\': 4", str(delivery))
            if four_match is not None:
                fours = fours + 1
            
            six_match = re.search(r"\'batsman\': 6", str(delivery))
            if six_match is not None:
                sixes = sixes + 1

            run_match = re.search(r"\'total\': (\d)", str(delivery))
            runs = int(run_match.group(1)) + runs_from_prev      # runs from previous illegal balls also included
            
            extra_match = re.search(r"\'extras\': (\d)", str(delivery))
            extras = int(extra_match.group(1)) + extras_from_prev   # extras from previous illegal balls also

            # cumulative runs and wickets
            runs_cume = runs_cume + runs
            wickets_cume = wickets_cume + wickets
            
            # get bowler
            bowler = "NA"
            bowler_match = re.search(r"\'bowler\': \'(.*?)\'", str(delivery))
            if bowler_match is not None:
                bowler = bowler_match.group(1)
            
            # update ball_no
            ball_no = ball_no + 1
            
            # add all to list
            innings_list.append([int(match_id), innings_name, int(ball_no), int(runs), int(wickets), int(runs_cume), 
                                 int(wickets_cume), team, bowler, int(fours), int(sixes), int(extras)])   

            # reset all at the end of a legal ball
            runs_from_prev = 0
            extras_from_prev = 0
            wickets = 0
            fours = 0
            sixes = 0

    inning_df = pd.DataFrame(innings_list, columns = ['match_id', 'innings', 'ball_no', 'runs', 'wicket', 
                                                      'runs_cume', 'wickets_cume', 'batting_team', 
                                                      'bowler', 'fours', 'sixes', 'extras'])
    return inning_df

In [35]:
# # for concatenating later
# match_df_list = []
# match_label_list = []

# for index, row in matches_df.iterrows():           # iterate all selected matches
    
#     if index % 25 == 0: 
#         print (index, "----", row['match_id'])     # keep track - REMOVE
    
#     with open('odis/' + str(row['match_id']) + '.yaml') as stream:
#         data_loaded = yaml.safe_load(stream)
        
# #         if str(row['match_id']) == '1193504':
# #             break;

#         # save ground, toss and match winner data
#         matches_df.at[index, 'ground'] = data_loaded['info']['venue']
#         if 'toss' in data_loaded['info']:
#             matches_df.at[index, 'toss_winner'] = data_loaded['info']['toss']['winner']
#             matches_df.at[index, 'toss_decision'] = data_loaded['info']['toss']['decision']
#         if 'winner' in data_loaded['info']['outcome']:
#             matches_df.at[index, 'match_winner'] = data_loaded['info']['outcome']['winner']
#             if 'method' in data_loaded['info']['outcome'] and data_loaded['info']['outcome']['method'] == "D/L":
#                 matches_df.at[index, 'dl_used'] = "y"
#         else:
#             matches_df.at[index, 'match_winner'] = "No Result"
        
#         # check number of innings and create empty dfs in case no balls bowled
#         max_innings = len(data_loaded['innings']) - 1
#         df_1st_innings = pd.DataFrame(columns = ['runs', 'wicket', 'runs_cume', 'wickets_cume'])
#         df_2nd_innings = pd.DataFrame(columns = ['runs', 'wicket', 'runs_cume', 'wickets_cume'])
        
#         # first innings
#         if (max_innings >= 0) and (data_loaded['innings'][0] is not None) and \
#             (data_loaded['innings'][0]['1st innings'] is not None) and \
#             (data_loaded['innings'][0]['1st innings']['deliveries'] is not None):
            
#             team = data_loaded['innings'][0]['1st innings']['team']
#             deliveries_1st_innings = data_loaded['innings'][0]['1st innings']['deliveries']
#             df_1st_innings = get_inning_df(deliveries_1st_innings, team, 'first_innings', row['match_id'])
#             if len(df_1st_innings) > 300:       # if either innings has more than 300 balls (miscounting), exclude
#                 continue
        
#         # second innings
#         if (max_innings >= 1) and (data_loaded['innings'][1] is not None) and \
#             (data_loaded['innings'][1]['2nd innings'] is not None) and \
#             (data_loaded['innings'][1]['2nd innings']['deliveries'] is not None):
            
#             team = data_loaded['innings'][1]['2nd innings']['team']
#             deliveries_2nd_innings = data_loaded['innings'][1]['2nd innings']['deliveries']    
#             df_2nd_innings = get_inning_df(deliveries_2nd_innings, team, 'second_innings', row['match_id'])
#             if len(df_2nd_innings) > 300:       # if either innings has more than 300 balls (miscounting), exclude
#                 continue
        
#         # concatenate both innings - one below the other
#         match_df = pd.concat([df_1st_innings, df_2nd_innings], axis = 0, sort = False)
        
#         # add these to list to be able to concatenate to df later
#         match_df_list.append(match_df)
#         match_label_list.append(str(row['match_id']))

# # concatenate all ball by ball info to a df
# full_df = pd.concat(match_df_list, axis = 0, sort = False)

# full_df['match_id'] = full_df['match_id'].astype(int)
# full_df['ball_no'] = full_df['ball_no'].astype(int)
# full_df['fours'] = full_df['fours'].astype(int)
# full_df['sixes'] = full_df['sixes'].astype(int)
# full_df['extras'] = full_df['extras'].astype(int)

# # take backup for later
# matches_backup_df = matches_df
# full_backup_df = full_df

# full_df.to_csv('full.csv', index = False)
# matches_df.to_csv('matches.csv', index = False)

Save the information to a csv for easier access later. 

In [36]:
matches_df = pd.read_csv("matches.csv", parse_dates = ['date'], 
                         date_parser = lambda x: pd.to_datetime(x, format = '%Y-%m-%d'))
full_df = pd.read_csv("full.csv")

In [37]:
# matches_df = matches_backup_df
# full_df = full_backup_df

### 2.3 Cleaning data
Now, removing duplicates.

In [38]:
all_grounds = matches_df.ground.unique()
len(all_grounds)

141

Looking throught the list of 141 grounds, it is found that few of them are repeated (with spelling mistakes or using older/alternate names). Moving onto correcting these.

In [39]:
matches_df['ground'].replace({'Darren Sammy National Cricket Stadium, Gros Islet': 'Beausejour Stadium, Gros Islet', 
                              'Darren Sammy National Cricket Stadium, St Lucia': 'Beausejour Stadium, Gros Islet', 
                              'Dubai Sports City Cricket Stadium': 'Dubai International Cricket Stadium', 
                              'Grange Cricket Club, Raeburn Place': 'Grange Cricket Club Ground, Raeburn Place', 
                              'ICC Global Cricket Academy': 'ICC Academy', 
                              'Kensington Oval, Barbados': 'Kensington Oval, Bridgetown', 
                              'National Cricket Stadium, Grenada': 'National Cricket Stadium, St George\'s', 
                              'Nehru Stadium, Fatorda': 'Nehru Stadium', 
                              'Providence Stadium, Guyana': 'Providence Stadium', 
                              'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association Stadium, Mohali', 
                              'Queen\'s Park Oval, Trinidad': 'Queen\'s Park Oval, Port of Spain', 
                              'R.Premadasa Stadium, Khettarama': 'R Premadasa Stadium', 
                              'Sharjah Cricket Association Stadium': 'Sharjah Cricket Stadium', 
                              'Vidarbha Cricket Association Stadium, Jamtha': 'Vidarbha Cricket Association Ground', 
                              'Zohur Ahmed Chowdhury Stadium': 'Zahur Ahmed Chowdhury Stadium',
                              'Civil Service Cricket Club, Stormont': 'Stormont Cricket Ground',
                              'Riverside Ground': 'County Ground',
                              'Kennington Oval': 'The Oval',
                              'Sophia Gardens': 'SWALEC Stadium',
                              'The Village, Malahide': 'Malahide Cricket Club Ground',
                              'Newlands': 'Sahara Park Newlands',
                              'St George\'s Park': 'Axxess DSL St. Georges',
                              'SuperSport Park': 'Centurion Park', 
                              'Shere Bangla National Stadium': 'Shere-e-Bangla Cricket Stadium',
                              'Green Park': 'Modi Stadium', 
                              'Maharani Usharaje Trust Cricket Ground': 'Holkar Cricket Stadium',
                              'Andhra Cricket Association-Visakhapatnam District Cricket Association Stadium': 
                                  'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
                              'OUTsurance Oval': 'Goodyear Park',
                              'Sedgars Park': 'North West Cricket Stadium', 
                              'Narayanganj Osmani Stadium': 'Khan Shaheb Osman Ali Stadium',
                              'Castle Avenue': 'Clontarf Cricket Club Ground',
                              'Feroz Shah Kotla': 'Arun Jaitley Stadium', 
                              'Chittagong Divisional Stadium': 'Zahur Ahmed Chowdhury Stadium',
                              'Indian Petrochemicals Corporation Limited Sports Complex Ground': 'Reliance Stadium',
                              'Jade Stadium': 'AMI Stadium',
                              'Chevrolet Park': 'Goodyear Park',
                              'Mangaung Oval': 'Goodyear Park'
                             }, inplace=True)

all_grounds = matches_df.ground.unique()
len(all_grounds)

116

We see that there are no such duplicates in team names.

In [40]:
full_df.batting_team.unique()

array(['Sri Lanka', 'New Zealand', 'Australia', 'South Africa', 'India',
       'Pakistan', 'West Indies', 'Bangladesh', 'England', 'Zimbabwe',
       'Ireland', 'Afghanistan'], dtype=object)

#### 2.3.1 Mapping grounds to country
Now, let us map the grounds to the countries. This will be done using data from wikipedia page "List of One Day International cricket grounds" at https://en.wikipedia.org/wiki/List_of_One_Day_International_cricket_grounds. The data was scraped and saved for later use. 

In [41]:
# html_object_page = urlopen("https://en.wikipedia.org/w/index.php?title=List_of_One_Day_International_cricket_grounds&oldid=945313988")         
# bs_obj_page = BeautifulSoup(html_object_page, "lxml")                      # get BS object
# grounds_table = bs_obj_page.find("table", {"class": "wikitable sortable"}) # go to the right table
# grounds_df = pd.read_html(str(grounds_table))[0]                           # convert html to dataframe

# grounds_df.to_csv("grounds_wiki.csv", index = False)

grounds_df = pd.read_csv("grounds_wiki.csv")

grounds_df.head()

,No.,Official name (known as),City,Country,First match,Last match,Matches
0,1,Melbourne Cricket Ground (MCG),Melbourne,Australia,5 January 1971,18 January 2019 [2],149 [3]
1,2,Old Trafford Cricket Ground,Manchester,England,24 August 1972,9 July 2019 [4],52 [5]
2,3,Lord's,London,England,26 August 1972,14 July 2019 [6],66 [7]
3,4,Edgbaston Cricket Ground,Birmingham,England,28 August 1972,11 July 2019 [8],63 [9]
4,5,AMI Stadium (formerly Jade Stadium and Lancast...,Christchurch,New Zealand,11 February 1973,29 January 2011 [10],48 [11]


Now that we have information about Stadiums, let us clean it up in preparation for matching with the grounds info from ball by ball data. The same cleaning up process to be done for ground info from ball by ball data as well. 

In [42]:
grounds_df.drop(['No.', 'City', 'First match', 'Last match', 'Matches'], axis = 1, inplace = True)
grounds_df.rename(columns = {"Official name (known as)": "ground_name", "Country": "country"}, inplace = True)

# remove all data in parantheses, and words like Stadium, Ground, Park, Cricketa
grounds_df['ground_name_adj'] = grounds_df['ground_name'].str.replace(r"\(.*\)","") 
grounds_df['ground_name_adj'] = grounds_df['ground_name_adj'].str.replace(r"Ground","") 
grounds_df['ground_name_adj'] = grounds_df['ground_name_adj'].str.replace(r"Stadium","") 
grounds_df['ground_name_adj'] = grounds_df['ground_name_adj'].str.replace(r"Cricket","") 
grounds_df['ground_name_adj'] = grounds_df['ground_name_adj'].str.strip()

grounds_df.head()

,ground_name,country,ground_name_adj
0,Melbourne Cricket Ground (MCG),Australia,Melbourne
1,Old Trafford Cricket Ground,England,Old Trafford
2,Lord's,England,Lord's
3,Edgbaston Cricket Ground,England,Edgbaston
4,AMI Stadium (formerly Jade Stadium and Lancast...,New Zealand,AMI


In [43]:
all_grounds_df = pd.DataFrame(all_grounds, columns = ['ground_name'])

# remove all data in parantheses, and words like Stadium, Ground, Park, Cricket
all_grounds_df['ground_name_adj'] = all_grounds_df['ground_name'].str.replace(r"\(.*\)","") 
all_grounds_df['ground_name_adj'] = all_grounds_df['ground_name_adj'].str.replace(r"\,(.*)","")
all_grounds_df['ground_name_adj'] = all_grounds_df['ground_name_adj'].str.replace(r"Ground","") 
all_grounds_df['ground_name_adj'] = all_grounds_df['ground_name_adj'].str.replace(r"Stadium","") 
all_grounds_df['ground_name_adj'] = all_grounds_df['ground_name_adj'].str.replace(r"Cricket","") 
all_grounds_df['ground_name_adj'] = all_grounds_df['ground_name_adj'].str.strip()

all_grounds_df.head()

,ground_name,ground_name_adj
0,AMI Stadium,AMI
1,Westpac Stadium,Westpac
2,"Brisbane Cricket Ground, Woolloongabba",Brisbane
3,Docklands Stadium,Docklands
4,Sydney Cricket Ground,Sydney


Now, to merge the dataframes using difflib such that small changes can also be caught. <br>
eg. 'MA Chidambaram' can be matched to 'M. A. Chidambaram' <br>
There are two National Stadiums - one in Pakistan and one in the Carribean. They can be handles manually. 

In [44]:
# we will use this as the merge key
all_grounds_df['merge_name'] = ''  

wiki_grounds_list = grounds_df['ground_name_adj'].to_list()

for index, row in all_grounds_df.iterrows():
    
    # there are two National Grounds - will handle manually at the end
    if row.ground_name_adj == "National":
        continue
        
    closest_match = 0.0                                          # default value of 0% match
    
    for ground_name_a in wiki_grounds_list:
        match_percentage = difflib.SequenceMatcher(None, ground_name_a, row.ground_name_adj).ratio()
        
        # only consider as match if no better match already available and this match is better than 85%
        if ((match_percentage > 0.50) & (match_percentage > closest_match)):          
            all_grounds_df.loc[index, 'merge_name'] = ground_name_a   
            closest_match = match_percentage                     # update best match percentage
        
        # break if perfect match available - it's not gonna get better and saves time
        if (closest_match == 1):                 
            break

# matching ground to country, dropping redundant columns, renaming columns
ground_country_df = pd.merge(left = all_grounds_df, right = grounds_df, how = 'left', 
                             left_on = 'merge_name', right_on = 'ground_name_adj')
ground_country_df.drop(['ground_name_adj_x', 'merge_name', 'ground_name_y', 'ground_name_adj_y'], 
                       axis = 1, inplace = True)
ground_country_df.rename(columns = {"ground_name_x": "ground_name"}, inplace = True)

# manually handling the two National Stadiums
ground_country_df.loc[ground_country_df['ground_name'] == 'National Cricket Stadium, St George\'s', 'country'] = \
                                                                                                'Grenada'
ground_country_df.loc[ground_country_df['ground_name'] == 'National Stadium', 'country'] = 'Pakistan'

# changing all Carribean countries to 'West Indies'
ground_country_df['country'].replace({'Grenada': 'West Indies', 
                                      'Barbados': 'West Indies', 
                                      'Trinidad & Tobago': 'West Indies', 
                                      'Guyana': 'West Indies', 
                                      'Antigua & Barbuda': 'West Indies',
                                      'Dominica': 'West Indies',
                                      'Jamaica': 'West Indies',
                                      'St Kitts & Nevis': 'West Indies',
                                      'Saint Vincent': 'West Indies'
                                     }, inplace = True)

# drop any duplicates that might have come up
ground_country_df.drop_duplicates(subset = ['ground_name', 'country'], keep = 'first', inplace = True)

ground_country_df.head(5)

,ground_name,country
0,AMI Stadium,New Zealand
1,Westpac Stadium,New Zealand
2,"Brisbane Cricket Ground, Woolloongabba",Australia
3,Docklands Stadium,Australia
4,Sydney Cricket Ground,Australia


Now, inputting these country names into matches_df

In [45]:
matches_df = pd.merge(left = matches_df, right = ground_country_df, how = 'left', 
                             left_on = 'ground', right_on = 'ground_name')
matches_df.drop(['ground'], axis = 1, inplace = True)
matches_df.rename(columns = {'country': 'ground_country'}, inplace = True)

matches_df.head()

,match_id,team_1,team_2,date,year,toss_winner,toss_decision,match_winner,dl_used,ground_name,ground_country
0,226391,New Zealand,Sri Lanka,2006-01-03,2006,New Zealand,field,New Zealand,NaN,AMI Stadium,New Zealand
1,226392,New Zealand,Sri Lanka,2006-01-06,2006,New Zealand,bat,New Zealand,NaN,Westpac Stadium,New Zealand
2,226375,Australia,South Africa,2006-01-15,2006,Australia,bat,South Africa,NaN,"Brisbane Cricket Ground, Woolloongabba",Australia
3,226383,South Africa,Sri Lanka,2006-01-17,2006,South Africa,field,Sri Lanka,NaN,"Brisbane Cricket Ground, Woolloongabba",Australia
4,226376,Australia,South Africa,2006-01-20,2006,Australia,bat,Australia,NaN,Docklands Stadium,Australia


All good here. 
## 3. Tracking Changing Trends Over the Years
First things first, we need to split the time period to observe trends. One obvious candidate is using the world cups to split the data into different periods. This might not give equal number of matches to each of the periods. Another option is to just select roughly equal number of matches. <br><br>
It would probably be better to go with the World Cup approach as World Cups sometimes signify significant rule changes as well as major changes in personnel. Both of these make World Cups very good points to split the data. <br><br>
This also means that there are very few matches after the 2019 World Cup (fewer than a 100) - therefore, removing those also from the analysis. <br>
The World Cup ending dates are:
* 28th April 2007
* 2nd April 2011
* 29th March 2015
* 14th July 2019

In [46]:
# naming the different time periods
matches_df.loc[matches_df['date'] < np.datetime64('2007-04-29'), 'split'] = 'WC03-WC07'
matches_df.loc[(matches_df['date'] < np.datetime64('2011-04-03')) & 
               (matches_df['date'] > np.datetime64('2007-04-29')), 'split'] = 'WC07-WC11'
matches_df.loc[(matches_df['date'] < np.datetime64('2015-03-30')) & 
               (matches_df['date'] > np.datetime64('2011-04-03')), 'split'] = 'WC11-WC15'
matches_df.loc[(matches_df['date'] < np.datetime64('2019-07-15')) & 
               (matches_df['date'] > np.datetime64('2015-03-30')), 'split'] = 'WC15-WC19'
matches_df.loc[(matches_df['date'] > np.datetime64('2019-07-15')), 'split'] = 'post2019'

matches_df.groupby('split').count()

,match_id,team_1,team_2,date,year,toss_winner,toss_decision,match_winner,dl_used,ground_name,ground_country
split,,,,,,,,,,,
WC03-WC07,180,180,180,180,180,180,180,180,14,180,180
WC07-WC11,413,413,413,413,413,413,413,413,34,413,413
WC11-WC15,427,427,427,427,427,427,427,427,47,427,427
WC15-WC19,312,312,312,312,312,312,312,312,28,312,312
post2019,17,17,17,17,17,17,17,17,3,17,17


We can see that all data has been split well. Also, post 2019 World Cup, these are only 17 matches. We will discard these from the analysis.

In [47]:
matches_df = matches_df[matches_df['split'] != 'post2019']

match_ids = matches_df['match_id'].unique().tolist()
full_df = full_df[full_df['match_id'].isin(match_ids)]

print(len(full_df['match_id'].unique()), len(match_ids))

1324 1332


Hmm. This is interesting. The number of unique matches in the full_df is less than the number from matches_df. This is because we did not add ball by ball details of matches where there were miscounting in balls into full_df. There would be no point in keeping details of those matches in matches_df either. Let us delete those.

In [48]:
match_ids = full_df['match_id'].unique().tolist()
matches_df = matches_df[matches_df['match_id'].isin(match_ids)]

print(len(full_df['match_id'].unique()), len(match_ids))

1324 1324


Good. In all, we are left with 1324 ODI matches from four different time periods for comparison. 

In [49]:
df = matches_df['split'].value_counts().rename_axis('time_period').reset_index(name = 'no_matches')
df = df.sort_values('time_period').reset_index().drop(['index'], axis = 1)

df

,time_period,no_matches
0,WC03-WC07,177
1,WC07-WC11,411
2,WC11-WC15,426
3,WC15-WC19,310


### 3.1 Scoring rate over the years
Let us first start off by having a look at the run rate and how that has changed over the years. 

In [50]:
# keep only run rate information at the end of innings and if more than 30 overs have been played (reduce outliers)
inning_rr_df = full_df.copy()
inning_rr_df['last_ball_no'] = \
        inning_rr_df.groupby(['match_id', 'innings'], sort = False)['ball_no'].transform(max)
inning_rr_df = inning_rr_df[(inning_rr_df['ball_no'] == inning_rr_df['last_ball_no']) & 
                            (inning_rr_df['last_ball_no'] > 180)]
inning_rr_df['run_rate'] = inning_rr_df['runs_cume'] / (inning_rr_df['ball_no'] / 6)
inning_rr_df.drop(inning_rr_df.columns[2:13], axis = 1, inplace = True)
inning_rr_df = pd.merge(left = inning_rr_df, right = matches_df[['match_id', 'split']], how = 'left', 
                             left_on = 'match_id', right_on = 'match_id')
inning_rr_df['run_rate'] = inning_rr_df['run_rate'].astype('float')

inning_rr_df.head()

,match_id,innings,run_rate,split
0,226391,first_innings,5.100000,WC03-WC07
1,226391,second_innings,5.333333,WC03-WC07
2,226392,first_innings,4.480000,WC03-WC07
3,226392,second_innings,4.350000,WC03-WC07
4,226375,first_innings,4.575251,WC03-WC07


In [51]:
fig = go.Figure()

fig.add_trace(go.Box(
    x = inning_rr_df.loc[inning_rr_df['innings'] == 'first_innings', 'split'],
    y = inning_rr_df.loc[inning_rr_df['innings'] == 'first_innings', 'run_rate'],
    name = 'First Innings',
    boxpoints = False,
    marker_color = '#1f77b4'
))


fig.add_trace(go.Box(
    x = inning_rr_df.loc[inning_rr_df['innings'] == 'second_innings', 'split'],
    y = inning_rr_df.loc[inning_rr_df['innings'] == 'second_innings', 'run_rate'],
    name = 'Second Innings',
    boxpoints = False,
    marker_color = '#ff7f0e'
))

fig.update_layout(boxmode = 'group', template = "plotly_white", title = "Run Rate Box Plots", 
                  yaxis = dict(title = "Run Rate"))
fig.show()

Excellent. There are two things to note here. <br><br>
One is that the **median run rates are going up over the years**. In each period considered, the median run rate has been higher that the previous. Before 2007 WC, the average run rate was under 5 an over while at present it is close to 5.5 an over. It is also clear that **the biggest jump was after 2015 WC**. This was the time when many ODI teams, led by England, chose a more aggresive approach. (It might be interesting to look at the run rate trends for the different teams - particularly England's)<br><br>
The other thing to note is that in the later years, higher scoring rates were acheived in the first innings. 
### 3.2 Scoring rate through the overs
Now, let us look at any changes in run scoring patterns at different stages of the game. Few rule changes in ODI cricket through the years and their impact on the scoring pattern can be explored. <br><br>
It is likely that matches affected by rain or any other interruptions will show different scoring patterns. For example, teams might look to accelerate earlier in a rain shortened game. Thus, we need to exclude matches that have been decided by Duckworth Lewis method as well as matches that were abandoned with no result. <br><br>
With those matches excluded, we can find the run rate and runs scored in each over and average it out for each of the time periods.

In [52]:
over_rr_df = full_df.copy()

# remove matches where there were rain interruptions - these might change scoring patterns
filtered_match_ids = matches_df.loc[(matches_df['dl_used'].isna()) & 
                                    (matches_df['match_winner'] != "No Result"), 'match_id']
over_rr_df = over_rr_df[over_rr_df['match_id'].isin(filtered_match_ids)]

# find run rate after each over and runs in each over
over_rr_df['run_rate'] = over_rr_df['runs_cume'] / (over_rr_df['ball_no'] / 6)
over_rr_df = over_rr_df[over_rr_df['ball_no'] % 6 == 0]
over_rr_df['runs_in_over'] = \
        np.where(over_rr_df['ball_no'] == 6, over_rr_df['runs_cume'], over_rr_df['runs_cume'].diff())
over_rr_df['over_no'] = (over_rr_df['ball_no'] / 6).astype(int)

# add split column and remove unnecessary columns
over_rr_df = pd.merge(left = over_rr_df, right = matches_df[['match_id', 'split']], how = 'left', 
                             left_on = 'match_id', right_on = 'match_id')
over_rr_df['run_rate'] = over_rr_df['run_rate'].astype('float')
over_rr_df['runs_in_over'] = over_rr_df['runs_in_over'].astype('float')
over_rr_df = over_rr_df.loc[:, ['split', 'match_id', 'innings', 'over_no', 'runs_in_over', 'run_rate']]

# group by overs and get average run-rate and runs per over 
over_rr_by_period_df = pd.DataFrame(over_rr_df.groupby([over_rr_df.split, 
                                                        over_rr_df.over_no])[['run_rate',
                                                                              'runs_in_over']].aggregate('mean'))
over_rr_by_period_df = pd.DataFrame(over_rr_by_period_df.to_records())

over_rr_by_period_df

,split,over_no,run_rate,runs_in_over
0,WC03-WC07,1,3.370968,3.370968
1,WC03-WC07,2,3.787097,4.203226
2,WC03-WC07,3,3.853763,3.987097
3,WC03-WC07,4,3.986290,4.383871
4,WC03-WC07,5,4.130323,4.706452
...,...,...,...,...
195,WC15-WC19,46,5.453591,7.803371
196,WC15-WC19,47,5.550785,8.416667
197,WC15-WC19,48,5.614514,8.170000
198,WC15-WC19,49,5.698158,8.947566


Excellent. Moving on to plotting and analysis.

In [53]:
period_list = over_rr_by_period_df['split'].unique().tolist()
traces = []

# loop through the four periods
for i in range(0, 4):
    df = over_rr_by_period_df[over_rr_by_period_df['split'] == period_list[i]]
    
    # line plot for run rate
    trace = go.Scatter(
        x = df['over_no'],
        y = df['run_rate'],
        name = 'Run Rate',
        visible = True if i == 0 else False,
        hovertemplate = "Average Run Rate After %{x} Overs: %{y:.3s}",
        marker_color = "#33aa4a"
    )
    traces.append(trace)
    
    # bar plot for runs in over
    trace = go.Bar(
        x = df['over_no'],
        y = df['runs_in_over'],
        name = 'Runs',
        hovertemplate = "Average Runs Scored in Over %{x}: %{y:.3s}",
        marker_color = "#bbeebb",
        visible = True if i == 0 else False,
        width = 0.8
    )
    traces.append(trace)

# set the above traces as data - there are 8 in all (2 visible at a time)
data = traces

# set up buttons
updatemenus = list([
    dict(active = 0,                           # WC03-WC07 is active
         type = "buttons", 
         direction = "down", 
         showactive = True,
         buttons = list([
            dict(label = "WC03-WC07", method = "update",
                 args = [{"visible": [True, True, False, False, False, False, False, False]}]),
            dict(label = "WC07-WC11", method = "update",
                 args = [{"visible": [False, False, True, True, False, False, False, False]}]),
            dict(label = "WC11-WC15", method = "update",
                 args = [{"visible": [False, False, False, False, True, True, False, False]}]),
            dict(label = "WC15-WC19", method = "update",
                 args = [{"visible": [False, False, False, False, False, False, True, True]}])
        ]),
    )
])

# general layout stuff
layout = dict(title = "Run Scoring Patterns Against Overs",
              xaxis = dict(title = "Overs"),
              yaxis = dict(title = "Runs", range = [0, 12]),
              template = "plotly_white",
              hovermode = "x",
              updatemenus = updatemenus)

# Create and show fig
fig = dict(data = data, layout = layout)
pio.show(fig)

There are many interesting patterns that can be observed here. The first thing being that the run rate at all stages of the game has been increasing over the years - we have already seen this earlier. Across the time periods, the scoring pattern is similar in that teams start slowly and are happy to steadily accumulate runs during the middle overs and accelerate towards the end. Still, there are certain things we can see. <br><br>
<b>WC03-WC07: </b>We can see two distinct places where the runs scored per over drops off from the previous overs. The first is coinciding with the end of fielding restrictions (at the end of 15 overs at the time) and the next is after 31 overs - when a new ball is taken. Batting teams are showing caution against the new ball. <br><br>
<b>WC07-WC11: </b>The only major change we can see here is that the runs per over drops off sharply at the end of 10 overs - the end of mandatory powerplay as per the new rules. It is followed by a relatively quiet period through overs 11-20. This is the period when usually the bowling powerplay was taken. It seems counter-intuitive that that would be a period of lower rate of scoring. <br><br>
<b>WC11-WC15: </b>Here also we can see the run rate dropping off after the mandatory 10 overs of powerplay. But, this was the era of the dreaded batting powerplay - the batting team gets to choose a block of 5 overs of fielding restrictions. This was usually taken towards the very end of the innings, particularly when batting first. This is reflected in the sharp increase in scoring in the last few overs. The increase is so stark that it is in fact more than the scoring rate in the last few overs of the WC15-WC19 time period as well. <br><br>
<b>WC15-WC19: </b>To correct for the batting powerplay, now all fielding restrictions are pre-determined. There are 5 fielders allowed outside the circle in the last 10 overs. But, only 4 through the middle overs. Also, there are two new balls being used from either end now - meaning that neither ball gets older than 25 overs at any point and thus taking reverse swing out of the equation. Both these changes has meant that the scoring rate through the middle overs has increased starkly. These changes were made with the express intention of livening up the middle overs. They seemed to have met their goal - but, possibly at the expense of competition between bat and ball. We had earlier seen that the jump in run rate from WC11-WC15 to WC15-WC19 was the highest. The connsistently higher rate of scoring all through the middle overs contributes to that greatly. <br>
### 3.3 Patterns in fall of wickets
Now, let us get an understanding of when wickets fall and also see how it might have changed over the years. Removing rain interrupted matches as they might skew when teams get more aggresive and lose wickets.

In [54]:
over_wkt_df = full_df.copy()

# remove matches where there were rain interruptions - these might change wicket fall patterns
filtered_match_ids = matches_df.loc[(matches_df['dl_used'].isna()) & 
                                    (matches_df['match_winner'] != "No Result"), 'match_id']
over_wkt_df = over_wkt_df[over_wkt_df['match_id'].isin(filtered_match_ids)]

# keep only rows with wickets and find their over number
over_wkt_df = over_wkt_df[over_wkt_df['wicket'] != 0]
over_wkt_df['over_no'] = np.ceil(over_wkt_df['ball_no'] / 6).astype(int)

# add split column and remove unnecessary columns
over_wkt_df = pd.merge(left = over_wkt_df, right = matches_df[['match_id', 'split']], how = 'left', 
                             left_on = 'match_id', right_on = 'match_id')
over_wkt_df = over_wkt_df.loc[:, ['split', 'match_id', 'innings', 'over_no', 'wickets_cume']]

over_wkt_df.head()

,split,match_id,innings,over_no,wickets_cume
0,WC03-WC07,226391,first_innings,6,1
1,WC03-WC07,226391,first_innings,28,2
2,WC03-WC07,226391,first_innings,33,3
3,WC03-WC07,226391,first_innings,38,4
4,WC03-WC07,226391,first_innings,42,5


In [55]:
fig = go.Figure()

fig.add_trace(go.Violin(x = over_wkt_df['split'][over_wkt_df['innings'] == 'first_innings' ],
                        y = over_wkt_df['over_no'][over_wkt_df['innings'] == 'first_innings' ],
                        name = 'First Innings', side = 'negative',
                        line_color = '#ff7f0e')
             )
fig.add_trace(go.Violin(x = over_wkt_df['split'][over_wkt_df['innings'] == 'second_innings' ],
                        y = over_wkt_df['over_no'][over_wkt_df['innings'] == 'second_innings' ],
                        name = 'Second Innings', side = 'positive',
                        line_color = '#1f77b4')
             )

fig.update_layout(violinmode = 'overlay', template = "plotly_white", title = 'Violin Plots - Loss of Wicket',
                  yaxis = dict(title = 'Overs', autorange = 'reversed'))

First of all, we need to address **two technical quirks in violin plots** that should be acknowledged before actually interpreting the plot. The plots starting before over 0 and extending beyong over 50 does not mean that there were wickets which fell before the innings started or after it ended. It is just a feature of kernal density estimation. So is the relative width of the plots for first and second innings. The second innings plot being wider does not suggest that more wickets fell in the second innigns - the density estimation functions for first and second innings are independent of each other. <br><br>
Now, something that stands out is the marked difference in when the wickets are lost depending on whether you are batting first or second. The plots suggest that **teams batting first tend to save wickets for later** and turn up the aggression (and thus lose wickets in the process) late in the innings. This could be to get a measure of how well the pitch is playing and to adjust accordingly. But, when batting second and having a target in front of them, there is less of a pressing need to conserve wickets while reading the pitch. This is reflected in wickets falling more evenly throughout the innings. <br><br>
Though the plots do not show much change across the different periods, something that can be seen is that **after WC 2015, there have been more wickets falling in the middle overs** as compared to earlier on in the innings. The rule changes to liven up the middle overs are clearly working. We had seen the same in the scoring patterns in [Section 3.2](#3.2-Scoring-rate-through-the-overs) as well. In any case, it is good to see that bowlers do not lose out entirely in the new order of things.
### 3.4 Wickets lost in innings
Now, let's take a look at the number of wickets that have fallen by the end of an innings. No need to remove the rain affected matches here as we do not need to look at overwise information - just removing matches with no results as those might include incomplete innings. 

In [56]:
last_wkt_df = full_df.copy()

filtered_match_ids = matches_df.loc[matches_df['match_winner'] != "No Result", 'match_id']
last_wkt_df = last_wkt_df[last_wkt_df['match_id'].isin(filtered_match_ids)]

# keep row only of last ball in each innings
last_wkt_df['last_ball_no'] = last_wkt_df.groupby(['match_id', 'innings'], sort = False)['ball_no'].transform(max)
last_wkt_df = last_wkt_df[(last_wkt_df['ball_no'] == last_wkt_df['last_ball_no'])]

# add split column and remove unnecessary columns
last_wkt_df = pd.merge(left = last_wkt_df, right = matches_df[['match_id', 'split']], how = 'left', 
                             left_on = 'match_id', right_on = 'match_id')
last_wkt_df = last_wkt_df.loc[:, ['split', 'match_id', 'innings', 'wickets_cume']]
last_wkt_df['wickets_cume'] = last_wkt_df['wickets_cume'].astype(int)

last_wkt_df.head()

,split,match_id,innings,wickets_cume
0,WC03-WC07,226391,first_innings,7
1,WC03-WC07,226391,second_innings,5
2,WC03-WC07,226392,first_innings,9
3,WC03-WC07,226392,second_innings,10
4,WC03-WC07,226375,first_innings,10


In [57]:
# aggregate to find avg wickets lost
last_wkt_by_period_df_1 = \
    pd.DataFrame(last_wkt_df.groupby([last_wkt_df.split, last_wkt_df.innings])['wickets_cume'].aggregate('mean'))
last_wkt_by_period_df_1 = pd.DataFrame(last_wkt_by_period_df_1.to_records())
last_wkt_by_period_df_1.rename(columns = {'wickets_cume': 'avg_wkts_lost'}, inplace = True)

# aggregate to find count of all outs
last_wkt_df_1 = last_wkt_df[last_wkt_df['wickets_cume'] == 10]
last_wkt_by_period_df_2 = \
    pd.DataFrame(last_wkt_df.groupby([last_wkt_df_1.split, last_wkt_df_1.innings])['wickets_cume'].aggregate('count'))
last_wkt_by_period_df_2 = pd.DataFrame(last_wkt_by_period_df_2.to_records())
last_wkt_by_period_df_2.rename(columns = {'wickets_cume': 'all_out_count'}, inplace = True)

# aggregate to find count of all
last_wkt_by_period_df_3 = \
    pd.DataFrame(last_wkt_df.groupby([last_wkt_df.split, last_wkt_df.innings])['wickets_cume'].aggregate('count'))
last_wkt_by_period_df_3 = pd.DataFrame(last_wkt_by_period_df_3.to_records())
last_wkt_by_period_df_3.rename(columns = {'wickets_cume': 'all_innings_count'}, inplace = True)

# merge all three and find count of not all outs 
last_wkt_by_period_df = pd.merge(left = last_wkt_by_period_df_1, right = last_wkt_by_period_df_2, how = 'left', 
                             left_on = ['split', 'innings'], right_on = ['split', 'innings'])
last_wkt_by_period_df = pd.merge(left = last_wkt_by_period_df, right = last_wkt_by_period_df_3, how = 'left', 
                             left_on = ['split', 'innings'], right_on = ['split', 'innings'])
last_wkt_by_period_df['avg_wkts_lost'] = last_wkt_by_period_df['avg_wkts_lost'].round(1)

last_wkt_by_period_df['not_all_out_count'] = last_wkt_by_period_df['all_innings_count'] - last_wkt_by_period_df['all_out_count']

last_wkt_by_period_df.head()

,split,innings,avg_wkts_lost,all_out_count,all_innings_count,not_all_out_count
0,WC03-WC07,first_innings,8.1,61,169,108
1,WC03-WC07,second_innings,6.5,54,169,115
2,WC07-WC11,first_innings,8.1,141,390,249
3,WC07-WC11,second_innings,6.8,145,390,245
4,WC11-WC15,first_innings,7.7,122,403,281


In [58]:
titles = ['WC03-WC07<br>First Innings', 'WC07-WC11<br>First Innings', 
          'WC11-WC15<br>First Innings', 'WC15-WC19<br>First Innings', 
          'WC03-WC07<br>Second Innings', 'WC07-WC11<br>Second Innings', 
          'WC11-WC15<br>Second Innings', 'WC15-WC19<br>Second Innings']
labels = ['All-out', 'Not all-out']
colors = ['#d4d4d4', '#bbeebb']
count = 0

specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}], 
         [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows = 2, cols = 4, specs = specs, 
                    subplot_titles = titles, horizontal_spacing = 0.01)


for index, df_row in last_wkt_by_period_df.iterrows():
    
    values = [df_row['all_out_count'], df_row['not_all_out_count']]
    
    col = math.floor(count / 2) + 1
    if (df_row['innings'] == 'first_innings'):
        row = 1
    else:
        row = 2
    
    fig.add_trace(go.Pie(labels = labels, values = values, 
                         marker_colors = colors, name = titles[count]), 
                  row = row, col = col)
    
    count = count + 1

fig2 = go.Figure(fig)
fig2.update_traces(hole=.33)
fig2.update_layout(title = "Proportion of All-out Innings")

for i in range(8):
    fig2.layout.annotations[i]["font"] = {'size': 12}

fig2.show()




fig1 = go.Figure()

fig1.add_trace(go.Bar(
    x = last_wkt_by_period_df.loc[last_wkt_by_period_df['innings'] == 'first_innings', 'split'],
    y = last_wkt_by_period_df.loc[last_wkt_by_period_df['innings'] == 'first_innings', 'avg_wkts_lost'],
    text = last_wkt_by_period_df.loc[last_wkt_by_period_df['innings'] == 'first_innings', 'avg_wkts_lost'],
    textposition = 'inside',
    textfont = dict(color = 'white'),
    name = 'First Innings',
    hoverinfo = 'skip',
    marker_color = "#1f77b4",
    width = 0.2
))
fig1.add_trace(go.Bar(
    x = last_wkt_by_period_df.loc[last_wkt_by_period_df['innings'] == 'second_innings', 'split'],
    y = last_wkt_by_period_df.loc[last_wkt_by_period_df['innings'] == 'second_innings', 'avg_wkts_lost'],
    text = last_wkt_by_period_df.loc[last_wkt_by_period_df['innings'] == 'second_innings', 'avg_wkts_lost'],
    textposition = 'inside',
    textfont = dict(color = 'white'),
    name = 'Second Innings',
    hoverinfo = 'skip',
    marker_color = "#ff7f0e",
    width = 0.2
))

fig1.update_layout(barmode = 'group', template = "plotly_white", title = "Average Wickets Lost Each Innings", 
                  bargap = 0.58, yaxis = dict(title = "Average Wickets Lost"))

fig1.show()

The donut charts show that more than **one third of all ODI innings end with the batting team all-out.** I found this extremely surprising - I had always believed that batsmen had it easier than this. In fact, across the two innings and for each era (except one), at least 30% the innings end with all ten batsmen back in the pavilion. <br><br>
We can see that **the proportion of innigns ending in all-out is higher in the second innings that the first.** This makes sense as there is no point in saving wickets if you are not close to chasing down the target. The best course of action would be to go harder at the target even at the cost of losing more wickets. It is intersting that after WC 2011, 40% of all second innings end with all wickets lost. After 2011 in fact, teams tend to get all out less often batting first and more often batting second as ealier.<br><br>
Looking at the bar graph of total wickets lost by the end of innings - one thing that is clear is that **teams lose more wickets batting first** (even though we already saw that teams tend to get all out more while batting second). This could be because you would expect almost half of the matches to end with the team batting second winning - and that inherently means that they have wickets left.<br><br>
It might be intersting to see how wickets lost change depending on if the team batting second won the match or lost it. In fact, let us do that now.
#### 3.4.1 Wicket in second innings by success of chase
First, to identify all matches which were won by teams chasing. 

In [59]:
win_lose_mat_df = matches_df.copy()
win_lose_mat_df = win_lose_mat_df[win_lose_mat_df['match_winner'] != 'No Result']
win_lose_mat_df['won_batting_first'] = (((win_lose_mat_df['toss_winner'] == win_lose_mat_df['match_winner']) & 
                                         (win_lose_mat_df['toss_decision'] == 'bat')) | 
                                        ((win_lose_mat_df['toss_winner'] != win_lose_mat_df['match_winner']) & 
                                         (win_lose_mat_df['toss_decision'] == 'field')))

last_wkt_df = full_df.copy()

filtered_match_ids = win_lose_mat_df.loc[:, 'match_id']
last_wkt_df = last_wkt_df[last_wkt_df['match_id'].isin(filtered_match_ids)]

# keep row only of last balls of second innings
last_wkt_df['last_ball_no'] = last_wkt_df.groupby(['match_id', 'innings'], sort = False)['ball_no'].transform(max)
last_wkt_df = last_wkt_df[(last_wkt_df['ball_no'] == last_wkt_df['last_ball_no']) & 
                          (last_wkt_df['innings'] == 'second_innings')]

# add split column and remove unnecessary columns
last_wkt_df = pd.merge(left = last_wkt_df, 
                       right = win_lose_mat_df[['match_id', 'split', 'won_batting_first']], 
                       how = 'left', left_on = 'match_id', right_on = 'match_id')
last_wkt_df = last_wkt_df.loc[:, ['split', 'match_id', 'innings', 'wickets_cume', 'won_batting_first']]
last_wkt_df['wickets_cume'] = last_wkt_df['wickets_cume'].astype(int)

last_wkt_df['won_chasing'] = ~(last_wkt_df['won_batting_first'])
last_wkt_df.drop(['won_batting_first'], axis = 1, inplace = True)

last_wkt_df.head()

,split,match_id,innings,wickets_cume,won_chasing
0,WC03-WC07,226391,second_innings,5,True
1,WC03-WC07,226392,second_innings,10,False
2,WC03-WC07,226375,second_innings,5,True
3,WC03-WC07,226383,second_innings,10,False
4,WC03-WC07,226376,second_innings,10,False


In [60]:
# aggregate to find avg wickets lost
last_wkt_by_period_df_1 = \
    pd.DataFrame(last_wkt_df.groupby([last_wkt_df.split, last_wkt_df.won_chasing])['wickets_cume'].aggregate('mean'))
last_wkt_by_period_df_1 = pd.DataFrame(last_wkt_by_period_df_1.to_records())
last_wkt_by_period_df_1.rename(columns = {'wickets_cume': 'avg_wkts_lost'}, inplace = True)

# aggregate to find count of all outs
last_wkt_df_1 = last_wkt_df[last_wkt_df['wickets_cume'] == 10]
last_wkt_by_period_df_2 = \
    pd.DataFrame(last_wkt_df.groupby([last_wkt_df_1.split, last_wkt_df_1.won_chasing])['wickets_cume'].aggregate('count'))
last_wkt_by_period_df_2 = pd.DataFrame(last_wkt_by_period_df_2.to_records())
last_wkt_by_period_df_2.rename(columns = {'wickets_cume': 'all_out_count'}, inplace = True)

# aggregate to find count of all
last_wkt_by_period_df_3 = \
    pd.DataFrame(last_wkt_df.groupby([last_wkt_df.split, last_wkt_df.won_chasing])['wickets_cume'].aggregate('count'))
last_wkt_by_period_df_3 = pd.DataFrame(last_wkt_by_period_df_3.to_records())
last_wkt_by_period_df_3.rename(columns = {'wickets_cume': 'all_innings_count'}, inplace = True)

# merge all three and find count of not all outs 
last_wkt_by_period_df = pd.merge(left = last_wkt_by_period_df_1, right = last_wkt_by_period_df_2, how = 'left', 
                             left_on = ['split', 'won_chasing'], right_on = ['split', 'won_chasing'])
last_wkt_by_period_df = pd.merge(left = last_wkt_by_period_df, right = last_wkt_by_period_df_3, how = 'left', 
                             left_on = ['split', 'won_chasing'], right_on = ['split', 'won_chasing'])
last_wkt_by_period_df['avg_wkts_lost'] = last_wkt_by_period_df['avg_wkts_lost'].round(1)

last_wkt_by_period_df['not_all_out_count'] = last_wkt_by_period_df['all_innings_count'] - last_wkt_by_period_df['all_out_count']

last_wkt_by_period_df

,split,won_chasing,avg_wkts_lost,all_out_count,all_innings_count,not_all_out_count
0,WC03-WC07,False,9.6,54.0,72,18.0
1,WC03-WC07,True,4.2,NaN,97,NaN
2,WC07-WC11,False,9.4,145.0,191,46.0
3,WC07-WC11,True,4.2,NaN,199,NaN
4,WC11-WC15,False,9.6,165.0,197,32.0
5,WC11-WC15,True,4.6,NaN,206,NaN
6,WC15-WC19,False,9.4,119.0,153,34.0
7,WC15-WC19,True,4.2,NaN,141,NaN


In [61]:
labels = ['All-out', 'Not all-out']
colors = ['#d4d4d4', '#bbeebb']
count = 0

specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows = 1, cols = 4, specs = specs, horizontal_spacing = 0.06, vertical_spacing = 0.0)


for index, df_row in last_wkt_by_period_df.iterrows():
    
    if (df_row.won_chasing == False):
        values = [df_row['all_out_count'], df_row['not_all_out_count']]

        col = count + 1
        row = 1

        fig.add_trace(go.Pie(labels = labels, values = values, title = df_row.split,
                             marker_colors = colors, name = df_row.split), 
                             row = row, col = col)
    
        count = count + 1

fig2 = go.Figure(fig)
fig2.update_traces(hole=.45)
fig2.update_layout(title = "Proportion of All-out Second Innings Among Matches Lost Chasing")


fig2.show()




fig1 = go.Figure()

fig1.add_trace(go.Bar(
    x = last_wkt_by_period_df.loc[last_wkt_by_period_df['won_chasing'] == True, 'split'],
    y = last_wkt_by_period_df.loc[last_wkt_by_period_df['won_chasing'] == True, 'avg_wkts_lost'],
    text = last_wkt_by_period_df.loc[last_wkt_by_period_df['won_chasing'] == True, 'avg_wkts_lost'],
    textposition = 'inside',
    name = 'Won Chasing',
    hoverinfo = 'skip',
    marker_color = "#bbeebb",
    width = 0.2
))
fig1.add_trace(go.Bar(
    x = last_wkt_by_period_df.loc[last_wkt_by_period_df['won_chasing'] == False, 'split'],
    y = last_wkt_by_period_df.loc[last_wkt_by_period_df['won_chasing'] == False, 'avg_wkts_lost'],
    text = last_wkt_by_period_df.loc[last_wkt_by_period_df['won_chasing'] == False, 'avg_wkts_lost'],
    textposition = 'inside',
    name = 'Lost Chasing',
    hoverinfo = 'skip',
    marker_color = "#ff9090",
    width = 0.2
))

fig1.update_layout(barmode = 'group', template = "plotly_white", title = "Average Wickets Lost in Second Innings", 
                  bargap = 0.58, yaxis = dict(title = "Average Wickets Lost"))

fig1.show()

Excellent. This clears up a lot - as expected, a **large proportion of second innings that are lost chasing ends with the team all out** (at least three fourths in each period). Nothing much to be seen period wise in the donut charts apart from that between WC 2011 and WC 2015, more second innings ended with all batsmen out when losing the chase than the other periods. <br><br>
And from the bar charts it is clear that teams hardly lose wickets when they win chasing - on average not even half of the wickets are lost. But, they **lose more than double that on average when they lose chasing**. In fact, the high averages seem to suggest that even when teams are not all-out, they are pretty close to. These figures have been remarkably consistent through the different periods - nothing to be gained from a period-wise comparison.
### 3.5 Batting team wise average and run rate
Let us see how different teams fared in their run scoring over the years. These are expected to reflect the quality of the batting lineups of each of the teams in each of the periods. 

In [62]:
team_wise_df = full_df.copy()

# keep row only of last ball in each innings
team_wise_df['last_ball_no'] = team_wise_df.groupby(['match_id','innings'], sort = False)['ball_no'].transform(max)
team_wise_df = team_wise_df[(team_wise_df['ball_no'] == team_wise_df['last_ball_no'])]

# add split column and remove unnecessary columns
team_wise_df = pd.merge(left = team_wise_df, right = matches_df[['match_id', 'split']], how = 'left', 
                        left_on = 'match_id', right_on = 'match_id')
team_wise_df = team_wise_df.loc[:, ['split', 'match_id', 'batting_team', 'ball_no', 'runs_cume', 'wickets_cume']]
team_wise_df['wickets_cume'] = team_wise_df['wickets_cume'].astype(int)

# aggregate, find average and run_rate averages
team_wise_by_period_df = team_wise_df.groupby(['split', 'batting_team']).agg({'ball_no':'sum',
                                                                              'runs_cume':'sum', 
                                                                              'wickets_cume':'sum'})
team_wise_by_period_df = pd.DataFrame(team_wise_by_period_df.to_records())
team_wise_by_period_df['average'] = team_wise_by_period_df['runs_cume'] / team_wise_by_period_df['wickets_cume']
team_wise_by_period_df['run_rate'] = team_wise_by_period_df['runs_cume'] / team_wise_by_period_df['ball_no'] * 6

# manually adding rows for Afghanistan
row = pd.DataFrame({'split': 'WC03-WC07', 'batting_team': 'Afghanistan'}, index = [0])
team_wise_by_period_df = pd.concat([row, team_wise_by_period_df.iloc[0:]], sort = False).reset_index(drop = True)
row = pd.DataFrame({'split': 'WC07-WC11', 'batting_team': 'Afghanistan'}, index = [12])
team_wise_by_period_df = pd.concat([team_wise_by_period_df.iloc[:12], row, 
                                 team_wise_by_period_df.iloc[12:]], sort = False).reset_index(drop = True)

# merge with aux data
teams_df = pd.read_csv('teams.csv')
team_wise_by_period_df = pd.merge(left = team_wise_by_period_df, right = teams_df, how = 'left', 
                                  left_on = 'batting_team', right_on = 'team')

team_wise_by_period_df.head()

,split,batting_team,ball_no,runs_cume,wickets_cume,average,run_rate,team,abbr
0,WC03-WC07,Afghanistan,NaN,NaN,NaN,NaN,NaN,Afghanistan,AFG
1,WC03-WC07,Australia,12863.0,11752.0,291.0,40.384880,5.481769,Australia,AUS
2,WC03-WC07,Bangladesh,5487.0,3783.0,167.0,22.652695,4.136687,Bangladesh,BAN
3,WC03-WC07,England,10061.0,8133.0,299.0,27.200669,4.850214,England,ENG
4,WC03-WC07,India,9132.0,7600.0,253.0,30.039526,4.993430,India,IND


In [63]:
# Create figure with secondary y-axis
fig = make_subplots(specs = [[{"secondary_y": True}]])
period_list = team_wise_by_period_df['split'].unique().tolist()

# loop through the four periods
for i in range(0, 4):
    df = team_wise_by_period_df[team_wise_by_period_df['split'] == period_list[i]]
    
    # line plot for run rate
    fig.add_trace(go.Bar(
        x = df['abbr'],
        y = df['average'],
        name = "Average", width = 0.5, marker = dict(color = "#5aceff"),
        visible = True if i == 0 else False
    ), secondary_y = False)
    
    # bar plot for runs in over
    fig.add_trace(go.Scatter(
        x = df['abbr'],
        y = df['run_rate'],
        name = "Run Rate", mode = 'markers', marker = dict(size = 20, color = "#00a7f9"),
        visible = True if i == 0 else False
    ), secondary_y = True)

# set up buttons
updatemenus = list([
    dict(active = 0,                           # WC03-WC07 is active
         type = "buttons", 
         direction = "down", 
         showactive = True,
         buttons = list([
            dict(label = "WC03-WC07", method = "update",
                 args = [{"visible": [True, True, False, False, False, False, False, False]}]),
            dict(label = "WC07-WC11", method = "update",
                 args = [{"visible": [False, False, True, True, False, False, False, False]}]),
            dict(label = "WC11-WC15", method = "update",
                 args = [{"visible": [False, False, False, False, True, True, False, False]}]),
            dict(label = "WC15-WC19", method = "update",
                 args = [{"visible": [False, False, False, False, False, False, True, True]}])
        ]),
    )
])

fig.update_layout(template = "plotly_white", updatemenus = updatemenus, hovermode = 'x', 
                  title = "Average and Run Rate by Batting Team")

fig.update_yaxes(title_text = "Average", range = [0, 60], showgrid = False, secondary_y = False)
fig.update_yaxes(title_text = "Run Rate",  range = [0, 7], showgrid = False, secondary_y = True)

fig.show()

Before going into interpreting the above data, something to acknowledge is that both **batting averages and run rate depend a lot on the grounds** that the matches are played at. Since it is reasonable to consider that each team would play a disproportionate amount of matches in their own country, how batting friendly their home pitches are could have skewed the data above. For example, countries like India and New Zealand are known for having batting friendly. It is worth noting that it is difficult to say how much of the batting average and run rate values we can attribute to the strength of the batting line and how much of it to attribute to the pitches. In any case, we shall also look at how the grounds in the countries affect scoring in the next section. <br><br>
**WC03-WC07:** This was the tail end of the era of Australian dominance. Both the average as well as the run rate of the Australian team stands head and shoulders above the rest. Till the end of 2015 World Cup, they remained the only team to average over 40. Notice also that Afghanistan has not started playing officially recognized ODI cricket yet and that there is a clear diffrence in quality between the top eight teams and the so called "minnows". <br><br>
**WC07-WC11:** Post the 2007 World Cup, many Australian greats retired. Now, India and South Africa have joined Australia at the top. There is hardly anything to choose between the top three teams at this point. <br><br>
**WC11-WC15:** Not a lot has changed here except that India and South Africa are putting some distance between themselves and Australia. Also, Afghanistan has started playing ODI matches and there is less to choose between the top eight teams and the "minnows". This could also have been because they were being excluded from the top flight of cricket and thus started playing each other a lot more. With slightly lower quality bowling, their batting stats could have improved<br><br>
**WC15-WC19:** There have been small jumps in averages and run rates over the last few periods, but nothing like the jump that we see post 2015 World Cup. And nowhere more so than England. While India also has comparable average, England scores at a much higher pace - for the first time in the history of cricket, a team has consistently scored at more than a run a ball for an extended period. This is also the first time a team from outside the top eight has scored at a higher average and pace than a team from the top eight (Bangladesh over West Indies and Sri Lanka).<br><br>
It is interesting to see how Bangladesh has risen over the years. In the beginning there is a large gulf between Bangladesh and the top eight teams. In each period, they have been improving their stats and post 2015 they have established themselves as one of the top teams. On the other hand both Sri Lanka and West Indies has falled behind the rest. 
#### 3.5.1 How grounds affect batting stats
Now, let us have a quick look at the batting same batting stats in each country to get an idea of how batting friendly each of the countries are. Not seperating this out on the time periods.

In [64]:
country_wise_df = full_df.copy()

# keep row only of last ball in each innings
country_wise_df['last_ball_no'] = country_wise_df.groupby(['match_id','innings'], 
                                                          sort = False)['ball_no'].transform(max)
country_wise_df = country_wise_df[(country_wise_df['ball_no'] == country_wise_df['last_ball_no'])]

# add split column and remove unnecessary columns
country_wise_df = pd.merge(left = country_wise_df, right = matches_df[['match_id', 'ground_country']], 
                           how = 'left', left_on = 'match_id', right_on = 'match_id')
country_wise_df = country_wise_df.loc[:, ['ground_country', 'ball_no', 'runs_cume', 'wickets_cume']]

# # aggregate, find average and run_rate averages
country_wise_df = country_wise_df.groupby('ground_country').agg({'ball_no':'sum', 'runs_cume':'sum', 
                                                                 'wickets_cume':'sum'})
country_wise_df = pd.DataFrame(country_wise_df.to_records())
country_wise_df['average'] = country_wise_df['runs_cume'] / country_wise_df['wickets_cume']
country_wise_df['run_rate'] = country_wise_df['runs_cume'] / country_wise_df['ball_no'] * 6

# remove data from countries except top 11 (no matches has happened in AFG)
top_teams = ['Ireland', 'West Indies', 'India', 'Afghanistan', 'Sri Lanka', 'Pakistan', 
             'Bangladesh', 'New Zealand', 'England', 'Australia', 'Zimbabwe', 'South Africa']
country_wise_df = country_wise_df[country_wise_df['ground_country'].isin(top_teams)]

# merge with aux data
teams_df = pd.read_csv('teams.csv')
country_wise_df = pd.merge(left = country_wise_df, right = teams_df, how = 'left', 
                           left_on = 'ground_country', right_on = 'team')

country_wise_df.head()

,ground_country,ball_no,runs_cume,wickets_cume,average,run_rate,team,abbr
0,Australia,80833,70894,2250,31.508444,5.262257,Australia,AUS
1,Bangladesh,59701,49094,1695,28.964012,4.933988,Bangladesh,BAN
2,England,88435,81012,2413,33.573145,5.496376,England,ENG
3,India,87246,78188,2339,33.427961,5.377072,India,IND
4,Ireland,11395,10274,313,32.824281,5.409741,Ireland,IRL


In [65]:
fig = make_subplots(specs = [[{"secondary_y": True}]])
    
# line plot for run rate
fig.add_trace(go.Bar(
    x = country_wise_df['abbr'],
    y = country_wise_df['average'],
    name = "Average", width = 0.5, marker = dict(color = "#bbeebb")), secondary_y = False)

# bar plot for runs in over
fig.add_trace(go.Scatter(
    x = country_wise_df['abbr'],
    y = country_wise_df['run_rate'],
    name = "Run Rate", mode = 'markers', marker = dict(size = 20, color = "#33aa4a")), secondary_y = True)

fig.update_layout(template = "plotly_white", hovermode = 'x', 
                  title = "Average and Run Rate by Host Country")

fig.update_yaxes(title_text = "Average", range = [0, 50], showgrid = False, secondary_y = False)
fig.update_yaxes(title_text = "Run Rate",  range = [0, 6], showgrid = False, secondary_y = True)

fig.show()

Big surprise. Matches in Pakistan show the highest average. The highest run rates are in New Zealand (smallish grounds) and England (who have hosted many high scoring matches since 2015). Can't think of a reason why Pakistani (who are known to have bowling friendly pitches, not to mention consistently strong bowling lineups) grounds return the best average. It could just be an abberation because of the smaller dataset we have for Pakistan - there have been no international cricket in Pakistan since 2009. <br><br>
In any case, there doesn't seem to be much we can gain from here. The stats here seem to correlate a lot with the batting strength of the home teams (who play close to half the innings in the country) rather than the quality of pitches - **most of the top teams are clustered around at 32-34 average** and the smaller teams tend to fall under that.
#### 3.5.2 Batting averages of teams by home and away
Another approach we can use to seperate the impact of home ground conditions on batting stats is to look at the home and away stats seperately. Let us do that briefly for the top teams - once again, not seperating by time periods. <br><br>
Here we are **considering all matches not in home country as away matches**. In cricketing nomenclature, away matches are only those matches that are home matches for the opposition - the rest are considered neutral. But, for the current use case, where we are looking at whether home ground conditions mask the true batting quality, it makes sense to consider all other matches as away matches. 

In [66]:
home_away_df = full_df.copy()

# keep row only of last ball in each innings
home_away_df['last_ball_no'] = home_away_df.groupby(['match_id','innings'], sort = False)['ball_no'].transform(max)
home_away_df = home_away_df[(home_away_df['ball_no'] == home_away_df['last_ball_no'])]

# add split column and remove unnecessary columns
home_away_df = pd.merge(left = home_away_df, right = matches_df[['match_id', 'ground_country']], how = 'left', 
                        left_on = 'match_id', right_on = 'match_id')
home_away_df = home_away_df.loc[:, ['batting_team', 'runs_cume', 'wickets_cume', 'ground_country']]
home_away_df['home_match'] = home_away_df['batting_team'] == home_away_df['ground_country']

# aggregate, find average and run_rate averages
home_away_df = home_away_df.groupby(['batting_team', 'home_match']).agg({'runs_cume':'sum', 'wickets_cume':'sum'})
home_away_df = pd.DataFrame(home_away_df.to_records())
home_away_df['average'] = home_away_df['runs_cume'] / home_away_df['wickets_cume']

# manually adding away row for AFG who have had no home matches
row = pd.DataFrame({'batting_team': 'Afghanistan', 'home_match': True}, index = [1])
home_away_df = pd.concat([home_away_df.iloc[:1], row, 
                          home_away_df.iloc[1:]], sort = False).reset_index(drop = True)

# merge with aux data
teams_df = pd.read_csv('teams.csv')
home_away_df = pd.merge(left = home_away_df, right = teams_df, how = 'left', 
                        left_on = 'batting_team', right_on = 'team')

home_away_df['average'] = home_away_df['average'].round(2)

home_away_df.head()

,batting_team,home_match,runs_cume,wickets_cume,average,team,abbr
0,Afghanistan,False,11456.0,498.0,23.00,Afghanistan,AFG
1,Afghanistan,True,NaN,NaN,NaN,Afghanistan,AFG
2,Australia,False,43981.0,1210.0,36.35,Australia,AUS
3,Australia,True,29828.0,818.0,36.46,Australia,AUS
4,Bangladesh,False,19822.0,722.0,27.45,Bangladesh,BAN


In [67]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = home_away_df.loc[home_away_df['home_match'] == True, 'abbr'],
    y = home_away_df.loc[home_away_df['home_match'] == True, 'average'],
    text = home_away_df.loc[home_away_df['home_match'] == True, 'average'],
    textposition = 'inside',
    textfont = dict(color = 'white'),
    textangle = 270,
    name = 'Home',
    marker_color = "#bbeebb",
    width = 0.35
))
fig.add_trace(go.Bar(
    x = home_away_df.loc[home_away_df['home_match'] == False, 'abbr'],
    y = home_away_df.loc[home_away_df['home_match'] == False, 'average'],
    text = home_away_df.loc[home_away_df['home_match'] == False, 'average'],
    textposition = 'inside',
    textfont = dict(color = 'white'),
    textangle = 270,
    name = 'Away',
    marker_color = "#ff9090",
    width = 0.35
))

fig.update_layout(barmode = 'group', template = "plotly_white", title = "Batting Average - Home and Away", 
                  bargap = 0.25, yaxis = dict(title = "Batting Average"))

fig.show()

Not a lot to learn from here either - apart from that most teams do much better at home than away. Ireland is the only team to buck the trend. 
### 3.6 Bowling average and economy rate
Now, similar to batting stats, let us look at the bowling stats to see which teams have had strong bowling lineups over the years.

In [68]:
bowling_df = full_df.copy()

# keep row only of last ball in each innings
bowling_df['last_ball_no'] = bowling_df.groupby(['match_id','innings'], sort = False)['ball_no'].transform(max)
bowling_df = bowling_df[(bowling_df['ball_no'] == bowling_df['last_ball_no'])]

# add split column and remove unnecessary columns
bowling_df = pd.merge(left = bowling_df, right = matches_df[['match_id', 'split', 'team_1', 'team_2']], 
                      how = 'left', left_on = 'match_id', right_on = 'match_id')
bowling_df = bowling_df.loc[:, ['split', 'match_id', 'batting_team', 'team_1', 'team_2', 
                                'ball_no', 'runs_cume', 'wickets_cume']]

# find bowling team
bowling_df['bowling_team'] = np.where((bowling_df['batting_team'] == bowling_df['team_1']), 
                                      bowling_df['team_2'], bowling_df['team_1'])

# aggregate, find average and run_rate averages
bowling_by_period_df = bowling_df.groupby(['split', 'bowling_team']).agg({'ball_no':'sum',
                                                                          'runs_cume':'sum', 
                                                                          'wickets_cume':'sum'})
bowling_by_period_df = pd.DataFrame(bowling_by_period_df.to_records())
bowling_by_period_df['average'] = bowling_by_period_df['runs_cume'] / bowling_by_period_df['wickets_cume']
bowling_by_period_df['econ_rate'] = bowling_by_period_df['runs_cume'] / bowling_by_period_df['ball_no'] * 6

# manually adding rows for Afghanistan
row = pd.DataFrame({'split': 'WC03-WC07', 'bowling_team': 'Afghanistan'}, index = [0])
bowling_by_period_df = pd.concat([row, bowling_by_period_df.iloc[0:]], sort = False).reset_index(drop = True)
row = pd.DataFrame({'split': 'WC07-WC11', 'bowling_team': 'Afghanistan'}, index = [12])
bowling_by_period_df = pd.concat([bowling_by_period_df.iloc[:12], row, 
                                  bowling_by_period_df.iloc[12:]], sort = False).reset_index(drop = True)

# merge with aux data
teams_df = pd.read_csv('teams.csv')
bowling_by_period_df = pd.merge(left = bowling_by_period_df, right = teams_df, how = 'left', 
                                left_on = 'bowling_team', right_on = 'team')

bowling_by_period_df.head()

,split,bowling_team,ball_no,runs_cume,wickets_cume,average,econ_rate,team,abbr
0,WC03-WC07,Afghanistan,NaN,NaN,NaN,NaN,NaN,Afghanistan,AFG
1,WC03-WC07,Australia,12881.0,10772.0,404.0,26.663366,5.017623,Australia,AUS
2,WC03-WC07,Bangladesh,5420.0,4160.0,132.0,31.515152,4.605166,Bangladesh,BAN
3,WC03-WC07,England,9283.0,8047.0,234.0,34.388889,5.201120,England,ENG
4,WC03-WC07,India,9458.0,7846.0,259.0,30.293436,4.977374,India,IND


In [69]:
# Create figure with secondary y-axis
fig = make_subplots(specs = [[{"secondary_y": True}]])
period_list = bowling_by_period_df['split'].unique().tolist()

# loop through the four periods
for i in range(0, 4):
    df = bowling_by_period_df[bowling_by_period_df['split'] == period_list[i]]
    
    # line plot for run rate
    fig.add_trace(go.Bar(
        x = df['abbr'],
        y = df['average'],
        name = "Average", width = 0.5, marker = dict(color = "#ff9090"),
        visible = True if i == 0 else False
    ), secondary_y = False)
    
    # bar plot for runs in over
    fig.add_trace(go.Scatter(
        x = df['abbr'],
        y = df['econ_rate'],
        name = "Economy Rate", mode = 'markers', marker = dict(size = 20, color = "#ff2828"),
        visible = True if i == 0 else False
    ), secondary_y = True)

# set up buttons
updatemenus = list([
    dict(active = 0,                           # WC03-WC07 is active
         type = "buttons", 
         direction = "down", 
         showactive = True,
         buttons = list([
            dict(label = "WC03-WC07", method = "update",
                 args = [{"visible": [True, True, False, False, False, False, False, False]}]),
            dict(label = "WC07-WC11", method = "update",
                 args = [{"visible": [False, False, True, True, False, False, False, False]}]),
            dict(label = "WC11-WC15", method = "update",
                 args = [{"visible": [False, False, False, False, True, True, False, False]}]),
            dict(label = "WC15-WC19", method = "update",
                 args = [{"visible": [False, False, False, False, False, False, True, True]}])
        ]),
    )
])

fig.update_layout(template = "plotly_white", updatemenus = updatemenus, hovermode = 'x', 
                  title = "Average and Run Rate by Batting Team")

fig.update_yaxes(title_text = "Average", range = [0, 60], showgrid = False, secondary_y = False)
fig.update_yaxes(title_text = "Economy Rate",  range = [0, 7], showgrid = False, secondary_y = True)

fig.show()

Once again, the previous caveat of pitches remain. Further, note that here lower average and economy rates are good. <br><br>
**WC03-WC07:** Similar to the batting analysis, here as well Australia shines. But, they are not alone - South Africa is also doing just as well or even better in the bowling department. Among the top teams, England have the poorest bowling average as well as economy rate. Unlike the batting case, there is no big divide between any of the top eight teams and the others except for Zimbabwe. <br><br>
**WC07-WC11:** Sri Lanka has joined Austalia and South Africa as one of the better bowling sides. Again, no big difference between the top teams and the others, particularly in economy rates.<br><br>
**WC11-WC15:** Here we see a large gap between Ireland and Zimbabwe and the rest. Even Afghanistan, a completely new side, is bowling at a higher level than them. South Africa has distanced themselves from the rest with a much better average while Pakistan has the best economy rate. They are the only team who conceded lesser than 5 runs an over. <br><br>
**WC15-WC19:** And now, Pakistan is one of the poorest performers, both in average and economy rate. Only Sri Lanka is behind them in either category. Sri Lanka, similar to their batting stats, have slipped up since 2015. The best bowling team by far is Afghanistan with the lowest average as well as economy rate and is the only side conceding lesser than 5 runs an over. Though Afghanistan does have quality bowlers, their stats probably has more to do with their oppositions than their bowling prowess. Afghanistan has been playing lower ranked teams for the most part during this period. <br><br>
### 3.7 Batting and bowling together

In [70]:
bowling_df = bowling_by_period_df[['team', 'abbr', 'split', 'average']]
batting_df = team_wise_by_period_df[['team', 'abbr', 'split', 'average']]

# remove no results and find loser for each match
win_lose_df = matches_df[matches_df['match_winner'] != 'No Result'].copy()
win_lose_df['match_loser'] = np.where(win_lose_df['team_1'] == win_lose_df['match_winner'], 
                                      win_lose_df['team_2'], win_lose_df['team_1'])

# aggregate to get number of wins and loses for each team
win_df = win_lose_df.groupby(['split', 'match_winner']).agg({'match_id':'count'})
lose_df = win_lose_df.groupby(['split', 'match_loser']).agg({'match_id':'count'})
win_df = pd.DataFrame(win_df.to_records())
lose_df = pd.DataFrame(lose_df.to_records())

# merge wins and loses data
win_lose_df = pd.merge(left = win_df, right = lose_df, how = 'left', 
                       left_on = ['split', 'match_winner'], right_on = ['split', 'match_loser'])
win_lose_df.drop(['match_loser'], axis = 1, inplace = True)
win_lose_df.rename(columns = {'match_winner': 'team', 'match_id_x': 'wins', 'match_id_y': 'loses'}, inplace = True)
win_lose_df['win_ratio'] = (win_lose_df['wins'] / win_lose_df['loses']).round(2)

# merge with batting and bowling averages
win_lose_df = pd.merge(left = win_lose_df, right = batting_df, how = 'left', 
                       left_on = ['split', 'team'], right_on = ['split', 'team'])
win_lose_df = pd.merge(left = win_lose_df, right = bowling_df, how = 'left', 
                       left_on = ['split', 'team'], right_on = ['split', 'team'])

# clean up dataframe structure
win_lose_df.drop(['wins', 'loses', 'abbr_y'], axis = 1, inplace = True)
win_lose_df.rename(columns = {'abbr_x': 'abbr', 
                              'average_x': 'batting_average', 
                              'average_y': 'bowling_average'}, 
                   inplace = True)
win_lose_df['batting_average'] = win_lose_df['batting_average'].round(2)
win_lose_df['bowling_average'] = win_lose_df['bowling_average'].round(2)

win_lose_df.head()

,split,team,win_ratio,abbr,batting_average,bowling_average
0,WC03-WC07,Australia,2.43,AUS,40.38,26.66
1,WC03-WC07,Bangladesh,0.62,BAN,22.65,31.52
2,WC03-WC07,England,0.59,ENG,27.20,34.39
3,WC03-WC07,India,0.74,IND,30.04,30.29
4,WC03-WC07,Ireland,0.29,IRL,19.49,31.33


In [71]:
fig = go.Figure()
period_list = win_lose_df['split'].unique().tolist()

# loop through the four periods
for i in range(0, 4):
    df = win_lose_df[win_lose_df['split'] == period_list[i]]
    
    # bar plot for runs in over
    fig.add_trace(go.Scatter(
        x = df['bowling_average'],
        y = df['batting_average'], 
        text = df['abbr'],
        textposition = 'top center',
        name = 'Imp Stats', 
        mode = 'markers+text', 
        hovertext = df['win_ratio'],
        hovertemplate = 'Win Ratio: %{hovertext}<br>Batting Avg: %{y}<br>Bowling Avg: %{x}',
        marker = dict(color = df['win_ratio'], size = 20, colorscale = 'Blues', 
                      colorbar = dict(thickness = 30, title = 'Win Ratio'), cmax = 2.5, cmin = 0),
        visible = True if i == 0 else False))

# set up buttons
updatemenus = list([
    dict(active = 0,                           # WC03-WC07 is active
         type = "buttons", 
         direction = "down", 
         x = 0.86,
         xanchor = "left",
         y = 1.0,
         yanchor = "top",
         showactive = True,
         buttons = list([
            dict(label = "WC03-WC07", method = "update",
                 args = [{"visible": [True, False, False, False]}]),
            dict(label = "WC07-WC11", method = "update",
                 args = [{"visible": [False, True, False, False]}]),
            dict(label = "WC11-WC15", method = "update",
                 args = [{"visible": [False, False, True, False]}]),
            dict(label = "WC15-WC19", method = "update",
                 args = [{"visible": [False, False, False, True]}])
        ]),
    )
])

fig.update_xaxes(range = [25, 45], title = 'Bowling Average')
fig.update_yaxes(range = [15, 45], title = 'Batting Average')
fig.update_layout(template = "plotly_white", updatemenus = updatemenus, 
                  margin = dict(t = 100, b = 0, l = 0, r = 0),
                  title = "Bowling and Batting Averages with Win Ratio")

fig.show()

The above shows scatter plots of teams according to batting and bowling averages. Top left corner is the ideal place to be with high batting average and low bowling average. Win ratio (i.e. number of wins per loss) is represented with colour. Mostly, top left corner does correlate well with higher win ratios. Also note that seeing run rate and economy rate also represented would provide a more complete picture than above.<br><br>
**WC03-WC07:** Australia was dominant in this period, followed by South Africa and Sri Lanka - at a distance. Australia's win ratio in this period is the highest any team has achieved within the scope of this analysis. England is some way behind with Bangladesh also having a better win ratio than England. Looks like their poor bowling performance was really hurting them. <br><br>
**WC07-WC11:** Again, Australia and South Africa are the leaders, but India also seems to be doing pretty well with the win ratio despite their weak bowling showing - their average is worst among the top eight teams. West Indies has started slipping to the bottom of the top eight pack. Ireland has only won one match for every 10 they have lost - the poorest return for any team across the four periods. <br><br>
**WC11-WC15:** There is no team with a win ratio over 2. This is the only such period among the four. Despite having the best bowling average (by a distance) and almost the best batting average, South Africa has won a smaller proportion of their matches than India and Australia. Once again, India's strong batting lineup is winning them matches despite a relatively weaker bowling performance. Sri Lanka is still slipping from when they were one of the top teams. <br><br>
**WC15-WC19:** Here there is much less correlation between top left and higher win ratios. England's strong batting (whose strength is not adequately represented here without a comparison of their run rate) is helping them win a lot more despite comparitively poorer bowling. It is interesting to see Pakistan towards the top right of the chart. The Pakistani team is usually associated with strong bowling and inconsistant batting. There are two clear clusters forming here (discounting Afghanistan) - teams that win more often than they lose and teams that lose more often than they win. And Bangladesh has put themselves firmly in the former - really great for a team that was still far from the top eight a little more than a decade ago. <br><br>
Couple more things that can be seen - the World Cup winners (Australia in 2007 & 2015, India in 2011 and England in 2019) were one of the top performing teams by win ratio. India in 2011 was the only slight exception to this. It is good that the **World Cups have been able to find the better teams of each period** despite condensing four years worth of cricket to little over a month. Similarly the runners up in 2007, 2011 and 2015 have been in the top three (Sri Lanka, Sri Lanka and New Zealand respectively) but in 2019, the runner up New Zealand was placed towards the middle of the pack. <br><br>
Another important trend to notice was how win ratio correlated strongly with the top left corner in the earlier periods, but has done so lesser and lesser later on. In WC07-WC11, India shows a better win ratio than teams like Sri Lanka, New Zealand, England etc. Similarly in WC11-WC15, India and Australia return better win ratios than South Africa who has a much better bowling average. In WC15-WC19, bowling average seems much less important than ever before. This trend is a strong indicator that **how well a team bowls is becoming lesser of a factor in determining how often they win** as compared to how well a team bats. It is disappointing that cricket is becoming less about bat vs ball. 
### 3.8 Toss to results
Let us also take a quick, almost superficial, look at if winning the toss is getting more advantageous or not over the years. We do not even need to look at the ball-by-ball dataset for this - just the matches data.

In [72]:
toss_decision_df = pd.DataFrame(matches_df.groupby([matches_df.split, 
                                                    matches_df.toss_decision])['match_id'].aggregate('count'))
toss_decision_df = pd.DataFrame(toss_decision_df.to_records())

toss_decision_df = toss_decision_df.pivot(index = 'split', columns = 'toss_decision', values = 'match_id')
toss_decision_df = pd.DataFrame(toss_decision_df.to_records())

toss_decision_df

,split,bat,field
0,WC03-WC07,103,74
1,WC07-WC11,242,169
2,WC11-WC15,200,226
3,WC15-WC19,155,155


In [73]:
toss_to_match_df = matches_df.loc[matches_df['match_winner'] != 'No Result', :].copy()
toss_to_match_df['toss_to_match'] = np.where(toss_to_match_df['toss_winner'] == toss_to_match_df['match_winner'], 
                                             'win', 'lose')

toss_to_match_df = pd.DataFrame(toss_to_match_df.groupby([toss_to_match_df.split, 
                                                          toss_to_match_df.toss_to_match])['match_id'].aggregate('count'))
toss_to_match_df = pd.DataFrame(toss_to_match_df.to_records())

toss_to_match_df = toss_to_match_df.pivot(index = 'split', columns = 'toss_to_match', values = 'match_id')
toss_to_match_df = pd.DataFrame(toss_to_match_df.to_records())

toss_to_match_df

,split,lose,win
0,WC03-WC07,83,86
1,WC07-WC11,201,189
2,WC11-WC15,211,192
3,WC15-WC19,155,139


In [74]:
labels = ['Bat', 'Field']
colors = ['#5aceff', '#bbeebb']
count = 0

specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows = 1, cols = 4, specs = specs, horizontal_spacing = 0.06, vertical_spacing = 0.0)


for index, df_row in toss_decision_df.iterrows():
    
    values = [df_row['bat'], df_row['field']]

    col = count + 1
    row = 1

    fig.add_trace(go.Pie(labels = labels, values = values, title = df_row.split,
                         marker_colors = colors, name = df_row.split, sort = False), 
                         row = row, col = col)

    count = count + 1

fig2 = go.Figure(fig)
fig2.update_traces(hole = .45)
fig2.update_layout(title = "Decision at Toss")

fig2.show()

In the first two periods, captains chose to bat more often than to field. But, after 2011 World Cup, the proportion is getting closer to even.

In [75]:
labels = ['Win', 'Lose']
colors = ['#bbeebb', '#ff9090']
count = 0

specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows = 1, cols = 4, specs = specs, horizontal_spacing = 0.06, vertical_spacing = 0.0)


for index, df_row in toss_to_match_df.iterrows():
    
    values = [df_row['win'], df_row['lose']]

    col = count + 1
    row = 1

    fig.add_trace(go.Pie(labels = labels, values = values, title = df_row.split,
                         marker_colors = colors, name = df_row.split, sort = False), 
                         row = row, col = col)

    count = count + 1

fig2 = go.Figure(fig)
fig2.update_traces(hole = .45)
fig2.update_layout(title = "Wins and Losses after Winning Toss")

fig2.show()

Counter-intuitively, it seems that teams lose more matches than they win on winning the toss. It also looks like the proportion is only increasing. But, it's all close enough to even that not a whole lot should be read into this. 